In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns

In [ ]:
path = '../input/mushroom-classification/mushrooms.csv'

In [ ]:
data= pd.read_csv(path)

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
unique_data= [] 
for x in data.columns.tolist():
    num= data[x].nunique()
    value= data[x].unique()
    unique_data.append(pd.Series({'col_name' : x,
               'num_unique' : num,
               'value_unique': value}))
uniques = pd.concat(unique_data,axis=1).T.set_index('col_name')
uniques

Getting all the unique values in the data.

In [ ]:
def missing_values(data):
    null_count = data.isnull().sum()
    empty_count = ((data == '') | (data== ' ')).sum()
    nan_count = ((data == 'nan') | (data== 'NaN')).sum()
    null_percent = null_count/ data.shape[0]
    nan_percent = nan_count/ data.shape[0]
    empty_percent = empty_count/ data.shape[0]
    value= pd.DataFrame({'null_count': null_count ,
                         'null_percent' : null_percent,
                         'nan_count': nan_count,
                         'nan_percent' : nan_percent,
                         'empty_count': empty_count,
                         'empty_percent' : empty_percent
                        })
    return value
missing_values(data)

There are 23 columns and 8124 rows, out of which none are empty/ nan/ null.

In [ ]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()
for x in data.columns.tolist():
    data[x]=le.fit_transform(data[x])
    print(x,le.classes_)

In [ ]:
data.head()

In [ ]:
#finding the correlation(absolute value)
cor= data.corr().abs()
cor

In [ ]:
plt.figure(figsize=(19,15))
ax= sns.heatmap(cor, annot= True)

In [ ]:
fig= plt.figure(figsize=(13,8))
ax= fig.gca()
data.hist(ax=ax)
plt.show()

In [ ]:
x_dropped_columns = data.drop(['cap-shape','cap-color','veil-type','habitat','class'],axis=1)  #removing some features

x_all_columns = data.drop('class', axis=1)   #taking all the features

y= data['class']

In [ ]:
from sklearn.model_selection import train_test_split

x_train_drop,x_test_drop,y_train_drop,y_test_drop = train_test_split(x_dropped_columns,y, test_size= 0.3, random_state=0)

x_train_all,x_test_all,y_train_all,y_test_all = train_test_split(x_all_columns, y, test_size= 0.3, random_state=0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
result= []
RF_1 = RandomForestClassifier(oob_score=False, 
                            random_state=42, 
                            warm_start=True,
                            n_jobs=-1)
RF_2 = RandomForestClassifier(oob_score=False, 
                            random_state=42, 
                            warm_start=True,
                            n_jobs=-1)
# Iterate through all of the possibilities for number of trees
for n_trees in [15, 20, 30, 40, 50, 100, 150, 200, 300, 400]:
    
    x1= RF_1.set_params(n_estimators=n_trees)     # Use this to set the number of trees
    x2= RF_2.set_params(n_estimators=n_trees)
    
    RF_1.fit(x_train_drop, y_train_drop)            # Fit the model
    RF_2.fit(x_train_all, y_train_all) 
    
    rf1_ypred = RF_1.predict(x_test_drop)
    rf2_ypred = RF_2.predict(x_test_all)
    
    result.append(pd.Series({'num_trees' : n_trees,
                            'pred_score_1' : accuracy_score(y_test_drop,rf1_ypred),
                            'pred_score_2' : accuracy_score(y_test_all,rf2_ypred)
                            }))

rf_result = pd.concat(result, axis=1).T.set_index('num_trees')  
rf_result

Surprisingly, it works with every number of trees we choose!

(If something is wrong, please let me know!)


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix_1 = confusion_matrix(y_test_drop, rf1_ypred)
confusion_matrix_2 = confusion_matrix(y_test_all, rf2_ypred)

print('confusion_matrix_1 \n', confusion_matrix_1)
print('\n')
print('confusion_matrix_2 \n', confusion_matrix_2)